In [1]:
import numpy as np
import warnings
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from deeptime.models.multiclass.inception import InceptionTimeClassifier
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset

C:\Users\medei\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.filterwarnings('ignore')

In [3]:
DATASET = 'ECG5000'
ACTIVATION = 'relu'
NUM_CLASSES = 5

VAL_SIZE = .15

In [4]:
x_train, y_train = load_UCR_UEA_dataset(name=DATASET, split='train')
# Since the features from the sktime are instatiated as objects we have to manually convert them
y_train = np.array(y_train, dtype=np.int32)
y_train = y_train - 1
y_train = np.eye(NUM_CLASSES)[y_train]

sequence_length = x_train.values[0][0].shape[0]

x_test, y_test = load_UCR_UEA_dataset(name=DATASET, split='test')
y_test = np.array(y_test, dtype='int32') # Fixing the labels type
y_test = y_test - 1
y_test = np.eye(NUM_CLASSES)[y_test]

x_train_transformed = []
for val in x_train.values:
    x_train_transformed.append(val[0].tolist())
x_train = np.array(x_train_transformed)

x_test_transformed = []
for val in x_test.values:
    x_test_transformed.append(val[0].tolist())
x_test = np.array(x_test_transformed)

x_train = np.expand_dims(x_train_transformed, axis=1)
x_test = np.expand_dims(x_test_transformed, axis=1)

In [5]:
full_train_dataset = BaseDataset(x=x_train, y=y_train)
full_size = len(full_train_dataset)
train_dataset, val_dataset = torch.utils.data.random_split(
    full_train_dataset,
    [full_size - (int(full_size * VAL_SIZE)), (int(full_size * VAL_SIZE))]
)

train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

test_dataset = BaseDataset(x=x_test, y=y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [6]:
len(train_dataset), len(val_dataset), len(test_dataset)

(425, 75, 4500)

In [7]:
model = InceptionTimeClassifier(
    in_channels=1,
    n_classes=NUM_CLASSES,
    activation=ACTIVATION,
    learning_rate=5e-3,
)
model

InceptionTimeClassifier(
  (model): Sequential(
    (0): InceptionBlock(
      (activation_function): ReLU()
      (inception_1): Inception(
        (bottleneck): Sequential()
        (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
        (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
        (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
        (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
        (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (inception_2): Inception(
        (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
        (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), strid

In [8]:
class LitProgressBar(pl.callbacks.TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(
            disable=True,
        )
        return bar

In [9]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator='gpu',
    devices=-1,
    callbacks=[LitProgressBar()]
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 261 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
261 K     Trainable params
0         Non-trainable params
261 K     Total params
1.046     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████| 17/17 [00:00<00:00, 27.43it/s, loss=0.931, v_num=2, val_loss=0.918, train_loss=0.938]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████| 17/17 [00:00<00:00, 26.00it/s, loss=0.931, v_num=2, val_loss=0.918, train_loss=0.938]


In [10]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:  17%|██████████                                                 | 24/141 [00:00<00:01, 76.48it/s]

ValueError: pos_label=1 is not a valid label. It should be one of [0, 2]